In [1]:
import folium
import pandas as pd
import pickle
import os
import h3
from label_samples_time_hexa import label_samples
from vizualize import draw_hexagons, draw_all_boarders_for_time_bin, draw_hexagons_with_values, draw_barriers
from func import rename_time_bins, calc_dist_time_bin, normalize_distances, get_time_bin_hexagons, get_min_max_dist

In [2]:
matrix = pd.read_pickle("/home/jaro/BINP29/Project_Eran/1_dist_matrix/eucl_dist.pkl")

In [3]:
time_bins = 18
resolution = 3
same_age_range = True
df = label_samples("/home/jaro/BINP29/Project_Eran/", time_bins, resolution, same_age_range)
time_bins = rename_time_bins(df)
time_bins_hexagons = get_time_bin_hexagons(df)

In [9]:
# calculate the average distances between neighboring hexagons for each time bin with the given parameters
time_bins_dist = calc_dist_time_bin(df, matrix,
                                        10,
                                        True,
                                        False)

In [17]:
selected_time_bin = time_bins[6]
hexagons = time_bins_hexagons[selected_time_bin]
time_bin = time_bins_dist[selected_time_bin]
time_bin = normalize_distances(time_bin)

In [18]:
barrier_hex = {}
barrier_lines = {}
for pair in time_bin:
    distance = time_bin[pair]
    pair = list(pair)
    # check if the pair are direct neighbors
    if pair[0] in h3.k_ring_distances(pair[1], 1)[1]:
        # get the line between the two hexagons
        boundary1 = h3.h3_to_geo_boundary(pair[0])
        boundary2 = h3.h3_to_geo_boundary(pair[1])
        # get the pair of dots that the two hexagons share
        shared_boundary = frozenset([x for x in boundary1 if x in boundary2])
        barrier_lines[shared_boundary] = distance
    else:
        try:
            line = h3.h3_line(pair[0], pair[1])
        except:
            continue
        # exclude the first and last hexagon
        line = line[1:-1]
        # add all elements of line as a key to the dictionary and give it the value of the pair
        for hex in line:
            if hex in hexagons:
                continue
            if hex in barrier_hex:
                barrier_hex[hex].append(distance)
            else:
                barrier_hex[hex] = [distance]

# calc the average distance for each hexagon and round it to 2 decimal places
for hex in barrier_hex:
    barrier_hex[hex] = round(sum(barrier_hex[hex])/len(barrier_hex[hex]), 2)
    
print(f"Number of barrier lines: {len(barrier_lines)}")
print(f"Number of barrier hexagons: {len(barrier_hex)}")

Number of barrier lines: 58
Number of barrier hexagons: 601


In [19]:
m = draw_hexagons(hexagons, color = 'black')
#m = draw_all_boarders_for_time_bin(time_bin, m)
m = draw_hexagons_with_values(barrier_hex, m, threshold = 0.5)
m = draw_barriers(barrier_lines, m)
m

In [16]:
# create two hexagons using h3
hex1 = h3.geo_to_h3(37.3615593, -122.0553238, 3)
hex2 = h3.geo_to_h3(36.3615593, -122.0553238, 3)
print(h3.k_ring_distances(hex1, 1)[1])

hex1_dots = h3.h3_to_geo_boundary(hex1)
hex2_dots = h3.h3_to_geo_boundary(hex2)
# get the pair of dots that the two hexagons share
dot = [x for x in hex1_dots if x in hex2_dots]
dot
# m = draw_hexagons([hex1, hex2], color = 'blue')
# # draw the line between the two hexagons
# folium.PolyLine(dot, color = 'red').add_to(m)
# m




{'8329a9fffffffff', '832835fffffffff', '832830fffffffff', '832836fffffffff', '83291bfffffffff', '83291afffffffff'}


[(36.57421829680793, -121.7071569184514),
 (36.5177842407864, -122.49913324533739)]